# Quantitative data sanity check

I generated some quantitative data from my models.
Every time I am testing my test set for each of the models and I am computing PSNR, LPIPS and SSIM.
I want to check now if the generated data actually conforms to what I think it should look like.

I will answer some (and more) of the following questions:

- Is every dataset, model and size accounted for?
- Can i compute the summary statistics per model?
- Are the summary statistics reasonable?

In [1]:
import pandas as pd

df_single = pd.read_excel("./data/singleMetricsResult.xlsx")

In [7]:
df_single

,Unnamed: 0,model,dataset,size,view_idx,psnr,ssim,lpips
0,0,GaussianModel(\n (params): ParameterDict(\n ...,truck,low,0,"tensor(22.8543, device='cuda:0', dtype=torch.f...","tensor(0.7905, device='cuda:0')","tensor([[[[0.2261]]]], device='cuda:0')"
1,1,GaussianModel(\n (params): ParameterDict(\n ...,truck,low,1,"tensor(22.0874, device='cuda:0', dtype=torch.f...","tensor(0.7851, device='cuda:0')","tensor([[[[0.2295]]]], device='cuda:0')"
2,2,GaussianModel(\n (params): ParameterDict(\n ...,truck,low,2,"tensor(20.9982, device='cuda:0', dtype=torch.f...","tensor(0.7453, device='cuda:0')","tensor([[[[0.2374]]]], device='cuda:0')"
3,3,GaussianModel(\n (params): ParameterDict(\n ...,truck,low,3,"tensor(22.4937, device='cuda:0', dtype=torch.f...","tensor(0.7999, device='cuda:0')","tensor([[[[0.1657]]]], device='cuda:0')"
4,4,GaussianModel(\n (params): ParameterDict(\n ...,truck,low,4,"tensor(23.0649, device='cuda:0', dtype=torch.f...","tensor(0.8157, device='cuda:0')","tensor([[[[0.1705]]]], device='cuda:0')"
...,...,...,...,...,...,...,...,...
2431,2431,GaussianModel(\n (params): ParameterDict(\n ...,stump,extended,11,"tensor(26.2844, device='cuda:0', dtype=torch.f...","tensor(0.7715, device='cuda:0')","tensor([[[[0.1466]]]], device='cuda:0')"
2432,2432,GaussianModel(\n (params): ParameterDict(\n ...,stump,extended,12,"tensor(26.9957, device='cuda:0', dtype=torch.f...","tensor(0.8161, device='cuda:0')","tensor([[[[0.1172]]]], device='cuda:0')"
2433,2433,GaussianModel(\n (params): ParameterDict(\n ...,stump,extended,13,"tensor(25.8797, device='cuda:0', dtype=torch.f...","tensor(0.7517, device='cuda:0')","tensor([[[[0.1529]]]], device='cuda:0')"
2434,2434,GaussianModel(\n (params): ParameterDict(\n ...,stump,extended,14,"tensor(26.7891, device='cuda:0', dtype=torch.f...","tensor(0.8123, device='cuda:0')","tensor([[[[0.1167]]]], device='cuda:0')"
